In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime, timedelta
import requests
import openpyxl
from pandas_market_calendars import get_calendar
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException, HTTPError, ConnectionError, Timeout

# Funções

In [2]:
def workday() -> datetime:
    return get_calendar('BMF').schedule(
        data_ini=(datetime.now() - timedelta(days=90)).strftime("%d/%m/%Y"),
        data_fim=datetime.now().strftime("%d/%m/%Y")
    ).loc[::-1, ("market_open")].iloc[0].date()

# Main

In [3]:
USER = Path(os.path.expanduser("~"))

In [4]:
excel_path = USER / r"C:\Users\GiordanoBrunoGava\OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx"
dados_cotacao__aj_consolidado_excel = pd.read_excel(excel_path)

In [5]:
dados_cotacao__aj_consolidado_excel

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,2024-02-15,"277,7","124,06","12,38","226,102418805","4,8",127804,"103,28",8778,15414,...,"92,8976217489","8,91286447742","87,1138471971","82,4769332469","8,7287315473","8,59162617668","11,0362010252","105,625387195","113,849902134","22,2779974546"
1,2024-02-16,"276,35","124,88","12,24",NaN,"4,88",128725,"104,01",8845,15332,...,"92,9905472483","8,94130978958","86,6538710642","82,4769332469","8,69082174688","8,59162617668","11,2518617079","105,798669821","113,353657799","22,2117502234"
2,2024-02-19,"276,7","125,14","12,16","229,772718325","4,98",129035,"103,73",8876,15341,...,"93,1206429475","8,9507915603","86,1888574094","82,4674857174","8,74768644751","8,59162617668","11,7206892791","107,146423577","113,076906151","22,3915641366"
3,2024-02-20,"272,3","126,19","12,05","226,122366085","4,8",129916,"105,85",8892,15063,...,"93,7618288933","8,9507915603","86,7392817355","82,4769332469","8,74768644751","8,61050887157","11,3044581828","107,541122891","112,924215586","22,694408622"
4,2024-02-21,"273,8","126,18","12,04","226,790599965","4,9",130031,"105,05",8847,15032,...,"93,808291643","9,06457280895","87,441547255","82,4769332469","8,7287315473","8,60106752413","11,4936541357","107,820300455","113,468175722","22,6849447318"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2024-08-06,"329,72","122,74","14,24",NaN,"5,81",126266,"98,4",8868,18481,...,"96,35","9,23","83,86","70,23","9,96","9,02","9,27","101,21","111,3","22,24"
121,2024-08-07,"325,81","124,04",14,NaN,"5,8",127513,"100,9",8882,18254,...,"96,11","9,28",83,"70,22","9,76","9,02","9,3","101,1","111,93","22,57"
122,2024-08-08,"330,72","125,3","14,29",NaN,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
123,2024-08-09,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"


In [6]:
dados_cotacao__aj_consolidado_excel['Data'] = pd.to_datetime(dados_cotacao__aj_consolidado_excel['Data'], format="%d/%m/%Y")

In [7]:
ultima_linha_com_data = dados_cotacao__aj_consolidado_excel['Data'].last_valid_index()

In [8]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_1=(datetime.now() - timedelta(days=360)).strftime("%d/%m/%Y")
data_fim_1=(datetime.now() - timedelta(days=300)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_1}%26data_fim%3D{data_fim_1}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_1 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [9]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_2=(datetime.now() - timedelta(days=299)).strftime("%d/%m/%Y")
data_fim_2=(datetime.now() - timedelta(days=240)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_2}%26data_fim%3D{data_fim_2}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_2 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [10]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_3=(datetime.now() - timedelta(days=239)).strftime("%d/%m/%Y")
data_fim_3=(datetime.now() - timedelta(days=180)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_3}%26data_fim%3D{data_fim_3}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_3 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [11]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_4=(datetime.now() - timedelta(days=179)).strftime("%d/%m/%Y")
data_fim_4=(datetime.now() - timedelta(days=120)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_4}%26data_fim%3D{data_fim_4}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_4 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [12]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_5=(datetime.now() - timedelta(days=119)).strftime("%d/%m/%Y")
data_fim_5=(datetime.now() - timedelta(days=60)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_5}%26data_fim%3D{data_fim_5}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_5 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [15]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_6=(datetime.now() - timedelta(days=59)).strftime("%d/%m/%Y")
data_fim_6=datetime.now().strftime("%d/%m/%Y") 

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_6}%26data_fim%3D{data_fim_6}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_6 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=6000)

In [16]:
data_1 = list()
for lin in (table := response_1.json()["tables"]["tab0"]):
    data_1.append(table[lin])

In [17]:
dados_cotacao__aj_novos_1 = pd.DataFrame(data_1)

In [18]:
data_2 = list()
for lin in (table := response_2.json()["tables"]["tab0"]):
    data_2.append(table[lin])

In [19]:
dados_cotacao__aj_novos_2 = pd.DataFrame(data_2)

dados_cotacao__aj_novos_2 = dados_cotacao__aj_novos_2.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_2.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,20/10/2023,"235,16","109,5","10,2","159,660558966","5,11",113155,"93,9",7675,12869,...,"88,358216147","8,64796352014","83,6405210454","79,097755923","8,29815037627","8,33804438328","9,96846577621","102,794012491","109,125255133","20,8394861491"
1,23/10/2023,"234,6","109,35","10,22","158,306490868","5,13",112784,"95,47",7660,12852,...,"87,9840434228","8,62989045113","84,4402131198","79,1067874092","8,20892295287","8,36505748603","9,96302843125","102,504056678","108,838715838","21,6344529232"
2,24/10/2023,"235,25","110,24","10,27","158,306490868","5,26",113761,"96,3",7705,12924,...,"88,2869451519","8,62989045113","84,9491080761","78,8990632271","8,22676843755","8,4460967943","9,94581017218","100,839149105","108,302610062","22,0792557611"
3,25/10/2023,"232,2","109,35","10,01","161,024583447","5,18",112829,"94,71",7654,12684,...,"87,9573167997","8,53952510609","84,6128739085","78,8990632271","8,25353666457","8,46410552946","9,94852884466","100,380831852","107,969854752","21,9089057381"
4,26/10/2023,229,"111,4","9,8","164,280320712","5,18",114776,"96,9",7764,12473,...,"87,5118730804","8,51241550258","84,6764857781","79,3867634806","8,18215472585","8,37406185362","9,94852884466","99,8196270523","106,426239844","22,3347807956"
5,27/10/2023,"228,99","109,85","9,91","163,424071768","5,22",113301,"94,3",7689,12480,...,"88,197856408","8,47626936457","84,8582339768","79,4770783423","8,22676843755","8,41908369154","10,1488043843","100,558546705","106,814454372","21,8899779577"
6,30/10/2023,"233,02","108,96","10,08","168,661129264","5,31",112531,"93,05",7652,12720,...,"88,0998587898","8,53048857159","84,6037864986","79,0255040336","8,29815037627","8,41908369154","9,78722094392","101,493888038","107,137966478","21,5871334724"
7,31/10/2023,234,"109,62","10,12","171,279658013","5,25",113143,"94,2",7667,12769,...,"88,1444031617","8,50337896808","84,5856116787","79,3416060497","8,33384134563","8,54872339995","9,78722094392","101,905438225","108,413528498","21,6723084839"
8,01/11/2023,"233,4","111,56","10,16","168,730823946","5,17",115052,"95,85",7811,12789,...,"87,8475668928","8,45868687832","84,830971747","78,8332387798","8,20757102221","8,61670164319","9,77815870231","101,886596506","108,022412865","21,624989033"
9,03/11/2023,"236,98","114,75","10,31","166,886344649","5,27",118159,100,7996,12964,...,"87,6675513869","8,51349478466","84,9127584364","79,523165789","8,19855171339","8,5432116505","9,92224834398","102,169222293","109,093324717","22,6092336105"


In [20]:
data_3 = list()
for lin in (table := response_3.json()["tables"]["tab0"]):
    data_3.append(table[lin])

In [21]:
dados_cotacao__aj_novos_3 = pd.DataFrame(data_3)

dados_cotacao__aj_novos_3 = dados_cotacao__aj_novos_3.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_3.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,19/12/2023,"257,2","128,05","11,4","236,072669196",5,131850,"112,48",8848,14034,...,"89,5716329238","8,74574826847","83,748486242","77,6967067582","8,28083153952","8,52846336009","10,2469736176","100,57179663","106,70780772","25,0225256032"
1,20/12/2023,"256,2","126,82","11,34","234,079401419","4,9",130804,"111,4",8811,14019,...,"90,00812413","8,71789556697","84,3706337726","78,0649375959","8,21692214506","8,62116404879","10,2460459539","100,638224501","106,867381324","24,558794985"
2,21/12/2023,"256,8","128,35","11,41","235,554419574","4,98",132182,"112,8",8897,14029,...,"90,0263112636","8,77360096996","83,9434877068","78,1569953053","8,22605205856","8,57481370444","10,2005904309","100,941894768","108,040716648","25,1739478459"
3,22/12/2023,"256,25","128,94","11,38","232,345258453","4,84",132752,"112,9",8948,13983,...,"90,4809896033","8,81073790528","85,3270695285","78,4239626626","8,28083153952","8,59335384218","10,3527272834","101,824436483","110,190266962","24,9846700425"
4,26/12/2023,"256,15","129,65","11,37","231,448287953","4,82",133532,"113,48",9006,13920,...,"90,4718960365","8,78288520379","84,5006347491","79,0683666287","8,326481107","8,63043411766","10,3620039208","102,298921275","111,297895508","25,2023395164"
5,27/12/2023,"256,21","130,46","11,41","233,720613219","4,83",134193,114,9040,13962,...,"90,7992644411","8,82002213911","85,2156401201","80,0902072033","8,34474093398","8,66751439314","10,3712805581","103,314318731","108,725944478","25,2307311869"
6,28/12/2023,"257,61","130,39","11,43","235,125867002","4,97",134185,"113,2",9073,13980,...,"90,7083287731","8,91286447742","85,7542155943","80,458438041","8,40865032845","8,65824432427","10,3712805581","103,342787819","110,640827727","25,126628395"
7,29/12/2023,,,,,,,,,,...,,,,,,,,,,
8,02/01/2024,259,"128,76","11,33","235,095967985","4,93",132696,"109,91",8994,14024,...,"91,4156240962","8,91286447742","86,4877925333","79,9450721371","8,63288100387","8,71811118217","10,3536549471","104,021416929","111,104314139","24,3695171816"
9,03/01/2024,"257,1","128,99","11,22","235,50458788","5,01",132833,"109,9",9015,13925,...,"91,2594679859","8,92224644002","86,5342214535","80,4677719666","8,56748039021","8,67134019943","10,4362170196","104,183905589","112,702869314","24,3695171816"


In [22]:
data_4 = list()
for lin in (table := response_4.json()["tables"]["tab0"]):
    data_4.append(table[lin])

In [23]:
dados_cotacao__aj_novos_4 = pd.DataFrame(data_4)

dados_cotacao__aj_novos_4 = dados_cotacao__aj_novos_4.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_4.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,19/02/2024,"276,7","125,14","12,16","229,772718325","4,98",129035,"103,73",8876,15341,...,"93,1206429475","8,9507915603","86,1888574094","82,4674857174","8,74768644751","8,59162617668","11,7206892791","107,146423577","113,076906151","22,3915641366"
1,20/02/2024,"272,3","126,19","12,05","226,122366085","4,8",129916,"105,85",8892,15063,...,"93,7618288933","8,9507915603","86,7392817355","82,4769332469","8,74768644751","8,61050887157","11,3044581828","107,541122891","112,924215586","22,694408622"
2,21/02/2024,"273,8","126,18","12,04","226,790599965","4,9",130031,"105,05",8847,15032,...,"93,808291643","9,06457280895","87,441547255","82,4769332469","8,7287315473","8,60106752413","11,4936541357","107,820300455","113,468175722","22,6849447318"
3,22/02/2024,"280,3","126,3","12,46","228,675617925",5,130240,"105,95",8882,15569,...,"93,7153661436","9,14990874543","87,2991961362","82,5808560723","8,7287315473","8,59162617668","11,5882521121","107,358213453","113,468175722","22,8931503155"
4,23/02/2024,"282,2","125,61","12,49","235,218325765","5,02",129418,"105,45",8817,15628,...,"93,9012171424","9,17835405759","87,4984877026","82,429695599","8,74768644751","8,62939156647","11,5693325168","107,338959828","112,609291297","22,694408622"
5,26/02/2024,"280,42","125,77","12,44","234,101278085","4,98",129609,"104,6",8815,15581,...,"93,9012171424","9,21628114048","87,32766636","82,136822182","8,81402859825","8,55386078689","11,3706767663","107,021275014","112,256194367","22,7606558531"
6,27/02/2024,"278,2","127,84","12,37","234,011515325",5,131689,"107,98",8939,15443,...,"94,0777755913","9,14042697471","87,5554281501","81,6738932325","8,90880309931","8,54441943945","11,1341818252","105,894937946","112,342082809","23,3852726042"
7,28/02/2024,"279,95","126,26","12,4",,"4,9",130155,"106,4",8871,15431,...,"94,3379669896","9,13094520399","87,8686006115","81,4566000521","8,95619034983","8,54441943945","11,0963426347","105,894937946","111,654975269","23,3852726042"
8,29/02/2024,"281,1","125,25","12,5",,"4,92",129020,"106,5",8834,15571,...,"95,2858070834","9,19731759904","88,0109517303","83,0248899627","8,95619034983","8,61050887157","11,1341818252","107,608510579","112,466143893","23,3663448239"
9,01/03/2024,"282,74","125,24","12,63","234,859274725","4,96",129180,"107,43",8839,15725,...,"94,8708649307","9,12067328571","88,3241241917","82,4703070284","9,09072539801","8,61622760671","10,7747095148","106,639064537","112,937357322","23,2527781419"


In [24]:
data_5 = list()
for lin in (table := response_5.json()["tables"]["tab0"]):
    data_5.append(table[lin])

In [25]:
dados_cotacao__aj_novos_5 = pd.DataFrame(data_5)

dados_cotacao__aj_novos_5 = dados_cotacao__aj_novos_5.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_5.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,17/04/2024,"292,4","120,54","12,79","205,835982324","5,18",124171,"99,85",8647,16375,...,"96,7114266168","9,11181077411","89,5026333692","71,2202064167","9,59183786734","8,7564918161","9,32192381815","104,323244857","114,019424049","20,6596722359"
1,18/04/2024,"292,3","120,48","12,66",,"5,24",124196,"99,07",8645,16287,...,"96,4331617719","9,10212765957","89,5220272225","70,9492599792","9,57236205441","8,74686929762","9,38954399093","103,786251084","113,360520264","20,4893222129"
2,19/04/2024,287,"121,52","12,36","198,954170724","5,18",125124,101,8734,15859,...,"97,0088821407","9,11181077411","89,696571903","70,5234870061","9,57236205441","8,78535937154","9,4571641637","103,669088806","113,27331241","20,5461055539"
3,22/04/2024,"288,29","121,8","12,39","202,963574004","5,25",125573,"101,11",8762,15968,...,"96,855356709","9,11181077411","89,5026333692","71,5105061711","9,61131368027","8,73724677914","9,37988396624","103,581217098","111,713260802","20,5650333342"
4,23/04/2024,"289,7","121,41","12,52","193,708036084","5,27",125148,"100,9",8667,16081,...,"96,9129287459","9,07307831598","88,435971433","71,2589130506","9,61131368027","8,73724677914","9,36056391688","103,493345389","111,529155333","20,6596722359"
5,24/04/2024,"289,73","120,97","12,58","187,763746644","5,33",124740,"99,98",8618,16012,...,"96,9033334064","9,11181077411","88,2032451924","69,6816177183","9,61131368027","8,68913418674","9,17702344792","102,321722611","110,996218448","20,4420027621"
6,25/04/2024,"290,51","120,91","12,54","188,362165044","5,42",124645,"99,4",8578,16264,...,"97,6325792068","9,08276143051","87,4177941303","70,2428639101","9,61131368027","8,66026663131","9,14804337388","102,272904996","110,124139909","19,4388304043"
7,26/04/2024,"289,8","122,83","12,62",,"5,46",126526,"101,76",8673,16200,...,"97,1048355355","9,11181077411","87,7087019311","70,1267440084","9,56262414794","8,64102159435","8,98382295428","102,243614426","110,928390117","20,3000444096"
8,29/04/2024,"290,95","123,77","12,67","188,182639524","5,52",127351,"102,85",8735,16257,...,"97,2391702882","9,12149388864","88,435971433","69,7590309861","9,54314833502","8,69875670522","8,94518285555","102,22408738","111,82953794","20,3473638604"
9,30/04/2024,291,"122,38","12,61","190,127499324","5,48",125924,"100,5",8679,16293,...,"97,1048355355","9,10212765957","87,4662787638","69,8945042048","9,65026530612","8,74686929762","9,0804232011","102,263141472","112,236507925","19,76060267"


In [26]:
data_6 = list()
for lin in (table := response_6.json()["tables"]["tab0"]):
    data_6.append(table[lin])

In [27]:
dados_cotacao__aj_novos_6 = pd.DataFrame(data_6)

dados_cotacao__aj_novos_6 = dados_cotacao__aj_novos_6.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_6.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,17/06/2024,"330,1","115,74","15,06",,"5,96",119137,93,8400,19171,...,"95,6283336288","9,18021505376","87,0526819709","66,595604067","9,7","8,82817074861","9,26385169928","100,975521971","108,812430174","20,65"
1,18/06/2024,"331,93","116,12","15,1",,"5,98",119630,"93,3",8463,19264,...,"95,7165154938","9,2","87,4493225084","64,7154904306","9,8","8,82817074861","9,26385169928","100,985369398","108,841955978","20,59"
2,19/06/2024,"332,75","116,76","15,23",,"5,94",120261,"93,49",8510,19432,...,"95,7263134788","9,2","87,4096584546","65,3092105263","9,76","8,84778890583","9,21441812564","100,768725993","109,934410731","20,61"
3,20/06/2024,"332,68","117,12","15,05","202,990265392","6,02",120445,94,8560,19254,...,"95,3931819887","9,27913978495","87,9451231803","65,6060705742","9,74","8,81836167","9,26385169928","99,8529152315","109,835991384","20,64"
4,21/06/2024,"330,85","118,02","14,95","199,952805229","5,94",121341,"94,39",8623,19140,...,"95,9810610889","9,21978494624","87,4790705487","65,2993151914","9,73","8,79874351278","9,36271884655","95,5298945425","109,058478542","20,83"
5,24/06/2024,327,"119,08","14,62",,"5,93",122636,"96,3",8712,18806,...,"95,2756061687","9,22967741935","87,6476427772","65,8138726077","9,66","8,82817074861","9,382492276","95,8154699411","108,910849521","21,3"
6,25/06/2024,332,"118,9","14,97",,"5,92",122331,96,8699,19208,...,"95,1384343786","9,20989247312","87,0130179171","65,0420364833","9,6","8,85759798444","9,47147270855","96,8691446876","110,229668773","21,28"
7,26/06/2024,"336,16","119,14","15,17",,6,122641,"96,12",8711,19489,...,"94,7955049035","9,25935483871","86,9733538634","65,6753379187","9,77","8,84778890583","9,3429454171","97,8046503037","112,286633128",21
8,27/06/2024,"336,25","120,8","15,18",,"5,86",124307,"97,8",8783,19477,...,"96,382778474","9,20989247312","87,3104983203","65,9820933014","9,79","8,84778890583","9,61977342945","98,2576319704","111,528804155","21,24"
9,28/06/2024,"339,83","120,38","15,32","210,01439202","5,96",123906,"96,45",8764,19721,...,"97,4899507793","9,22967741935","89,2242889138","66,18","9,85","8,88702522026","9,6","98,3561062458","109,855675254","21,12"


In [28]:
dados_cotacao__aj_novos = pd.concat([dados_cotacao__aj_novos_1, dados_cotacao__aj_novos_2, dados_cotacao__aj_novos_3, dados_cotacao__aj_novos_4, dados_cotacao__aj_novos_5, dados_cotacao__aj_novos_6], ignore_index=True) 

In [29]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(
    columns = {
        # Dict comprehension para colocar os nomes antigos da coluna com os valores da primeira linha.
        nome_antigo_da_coluna: nome_novo_da_coluna for
            nome_antigo_da_coluna, nome_novo_da_coluna in
                # Zip para poder fazer unpack das duas informacoes
                zip(
                    list(dados_cotacao__aj_novos.columns), [
                        # Como os dados da primeira linha vem em formatod indeseja ja formatamos aqui
                        str(item).replace("\nPRECO_AJ(,,,A,C)", "") for
                            item in
                                # iloc para pegar a primeira linha e todas as colunas
                                list(dados_cotacao__aj_novos.iloc[0, :])
                    ]
                )
        }
)

In [30]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(index = 0)

In [31]:
# ARRUMA AS DATAS ERRADAS PARA O FORMATO DAS OUTRAS
parser.parse("12-06-2024", dayfirst = True)

datetime.datetime(2024, 6, 12, 0, 0)

In [32]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,21/08/2023,"242,7","110,79","10,37","158,645007893","5,37",114429,"107,23",7717,13071,...,"88,6595181064","8,5392155673","82,1203141789","77,8407247633","7,82927091165","8,08937952347","9,73634717588","104,314184992","107,066266902","21,6723084839"
2,22/08/2023,"240,19","112,41","10,27",,"5,3",116156,"109,15",7821,12929,...,"88,1562638443","8,55685857467","82,0231826245","78,1923883042","7,74227901263","8,03633441184","9,657571276","102,647883034","105,366946591","22,2117502234"
3,23/08/2023,"238,65","114,3","10,29","151,227502503","5,27",118134,"110,55",7899,12902,...,"88,1649406419","8,5392155673","82,7384240706","78,2363462468","7,81187253185","8,04517526377","9,69386129729","102,960890032","104,676313737","22,8174391941"
4,24/08/2023,"236,8","113,3","10,09","150,082518449","5,3",117025,"108,32",7843,12740,...,"88,6508413087","8,52157255993","82,4293691248","78,2451378353","7,69878306312","8,04517526377","9,73634717588","103,108187443","105,357859316","22,8174391941"
5,25/08/2023,"238,35","112,15","10,16","146,796912035","5,31",115837,"107,25",7802,12810,...,"89,2408635471","8,54803707099","82,9856680273","78,1572219501","7,78577496214","8,03633441184","9,79742062634","103,82626232","107,039005079","22,4956669284"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,08/08/2024,"330,72","125,3","14,29",,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
245,09/08/2024,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"
246,12/08/2024,"327,45","127,65","14,2",,"5,83",131115,104,9093,18505,...,96,"9,17","84,4","68,95","9,75","9,04","9,32","102,3","112,19","23,05"
247,13/08/2024,"329,9","128,78","14,45",,"5,81",132397,104,9166,18761,...,"96,1","9,22","85,41","69,49","9,97","9,06","9,37","102,12","112,28","23,33"


In [33]:
dados_cotacao__aj_novos.reset_index(drop = True)

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,21/08/2023,"242,7","110,79","10,37","158,645007893","5,37",114429,"107,23",7717,13071,...,"88,6595181064","8,5392155673","82,1203141789","77,8407247633","7,82927091165","8,08937952347","9,73634717588","104,314184992","107,066266902","21,6723084839"
1,22/08/2023,"240,19","112,41","10,27",,"5,3",116156,"109,15",7821,12929,...,"88,1562638443","8,55685857467","82,0231826245","78,1923883042","7,74227901263","8,03633441184","9,657571276","102,647883034","105,366946591","22,2117502234"
2,23/08/2023,"238,65","114,3","10,29","151,227502503","5,27",118134,"110,55",7899,12902,...,"88,1649406419","8,5392155673","82,7384240706","78,2363462468","7,81187253185","8,04517526377","9,69386129729","102,960890032","104,676313737","22,8174391941"
3,24/08/2023,"236,8","113,3","10,09","150,082518449","5,3",117025,"108,32",7843,12740,...,"88,6508413087","8,52157255993","82,4293691248","78,2451378353","7,69878306312","8,04517526377","9,73634717588","103,108187443","105,357859316","22,8174391941"
4,25/08/2023,"238,35","112,15","10,16","146,796912035","5,31",115837,"107,25",7802,12810,...,"89,2408635471","8,54803707099","82,9856680273","78,1572219501","7,78577496214","8,03633441184","9,79742062634","103,82626232","107,039005079","22,4956669284"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,08/08/2024,"330,72","125,3","14,29",,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
244,09/08/2024,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"
245,12/08/2024,"327,45","127,65","14,2",,"5,83",131115,104,9093,18505,...,96,"9,17","84,4","68,95","9,75","9,04","9,32","102,3","112,19","23,05"
246,13/08/2024,"329,9","128,78","14,45",,"5,81",132397,104,9166,18761,...,"96,1","9,22","85,41","69,49","9,97","9,06","9,37","102,12","112,28","23,33"


In [34]:
dados_cotacao__aj_novos['Data'] = pd.to_datetime(dados_cotacao__aj_novos['Data'])

C:\Users\GiordanoBrunoGava\AppData\Local\Temp\ipykernel_34948\1604425695.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dados_cotacao__aj_novos['Data'] = pd.to_datetime(dados_cotacao__aj_novos['Data'])


In [35]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,2023-08-21,"242,7","110,79","10,37","158,645007893","5,37",114429,"107,23",7717,13071,...,"88,6595181064","8,5392155673","82,1203141789","77,8407247633","7,82927091165","8,08937952347","9,73634717588","104,314184992","107,066266902","21,6723084839"
2,2023-08-22,"240,19","112,41","10,27",,"5,3",116156,"109,15",7821,12929,...,"88,1562638443","8,55685857467","82,0231826245","78,1923883042","7,74227901263","8,03633441184","9,657571276","102,647883034","105,366946591","22,2117502234"
3,2023-08-23,"238,65","114,3","10,29","151,227502503","5,27",118134,"110,55",7899,12902,...,"88,1649406419","8,5392155673","82,7384240706","78,2363462468","7,81187253185","8,04517526377","9,69386129729","102,960890032","104,676313737","22,8174391941"
4,2023-08-24,"236,8","113,3","10,09","150,082518449","5,3",117025,"108,32",7843,12740,...,"88,6508413087","8,52157255993","82,4293691248","78,2451378353","7,69878306312","8,04517526377","9,73634717588","103,108187443","105,357859316","22,8174391941"
5,2023-08-25,"238,35","112,15","10,16","146,796912035","5,31",115837,"107,25",7802,12810,...,"89,2408635471","8,54803707099","82,9856680273","78,1572219501","7,78577496214","8,03633441184","9,79742062634","103,82626232","107,039005079","22,4956669284"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2024-08-08,"330,72","125,3","14,29",,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
245,2024-08-09,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"
246,2024-08-12,"327,45","127,65","14,2",,"5,83",131115,104,9093,18505,...,96,"9,17","84,4","68,95","9,75","9,04","9,32","102,3","112,19","23,05"
247,2024-08-13,"329,9","128,78","14,45",,"5,81",132397,104,9166,18761,...,"96,1","9,22","85,41","69,49","9,97","9,06","9,37","102,12","112,28","23,33"


In [36]:
if dados_cotacao__aj_consolidado_excel.index.duplicated().any():
    dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.reset_index(drop=True)
if dados_cotacao__aj_novos.index.duplicated().any():
    dados_cotacao__aj_novos = dados_cotacao__aj_novos.reset_index(drop=True)

In [37]:
dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.loc[:, ~dados_cotacao__aj_consolidado_excel.columns.duplicated()]
dados_cotacao__aj_novos = dados_cotacao__aj_novos.loc[:, ~dados_cotacao__aj_novos.columns.duplicated()]

In [38]:
# SALVA O DF CONCATENANDO NO EXCEL
output_path = USER / r'OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx'

dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(dados_cotacao__aj_novos.index[-1])
dados_cotacao__aj_novos.to_excel(output_path, index=False)

df = pd.read_excel(output_path)
df.replace(['-', '[]'], np.nan, inplace=True)
df = df.dropna(how='all', subset=df.columns.difference(['DATA']))
df.to_excel(output_path, index=False)


print('Dados atualizados e salvos no Excel.')
print('URL construída:', url + payload)

Dados atualizados e salvos no Excel.
URL construída: https://www.comdinheiro.com.br/Clientes/API/EndPoint001.phpusername=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D16/06/2024%26data_fim%3D14/08/2024%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3
